In [9]:
from music21 import *
from comp2 import *

In [10]:
melody_in: stream.Stream = parse_midi("example_inputs/deadoralive.mid")

print("Original melody:")
melody_in.show('midi');

Original melody:


In [11]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['i', '#iiio5b3', 'iv', 'III', 'III', '#iiio5b3', 'v', 'iv', 'III', 'III', 'v', 'III', 'bVII', 'III', 'III', 'i', 'v']


In [12]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')